# Capstone project

This notebook will be used for the capstone project of Coursera's IBM Data Science certification Capstone Project.

## Week 1 assignment

It will consist on basic initialization of the notebook.

In [1]:
# import pandas and numpy libraries
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Week 3 assignment, task 1

It will consist on getting a dataframe representing the different neighborhoods in Toronto.

First I will import the libraries required.

In [3]:
# import pandas and numpy libraries
import pandas as pd
import numpy as np

# library to handle requests
import requests 

# Library for getting latitude and longitude of an address
!pip install geocoder
import geocoder

# module to convert an address into latitude and longitude values
!pip install geopy
from geopy.geocoders import Nominatim

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# import k-means from clustering sklearn library
from sklearn.cluster import KMeans
    
# tranforming json file into a pandas dataframe library
from pandas import json_normalize

# Module for map plotting
! pip install folium==0.5.0
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Module for HTML scrapping
! pip install bs4
from bs4 import BeautifulSoup

     |████████████████████████████████| 102kB 4.4MB/s ta 0:00:011
     |████████████████████████████████| 112kB 5.4MB/s eta 0:00:01
     |████████████████████████████████| 122kB 4.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


Now I will request the data with the neighborhoods of Toronto from Wikipedia.

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_wikipedia_toronto = requests.get(url).text

Now we scrap the table of neighborhoods. I'll use BeautifulSoup for that, stripping carriage return characters and following the conditions fixed in the instructions.

In [5]:
soup_toronto = BeautifulSoup(html_wikipedia_toronto, 'html.parser')
html_toronto_table = soup_toronto.find("table", {"class":"wikitable sortable"})
toronto_neighborhoods = []
for items in html_toronto_table.find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        postalCode = data[0].text.strip('\n')
        borough = data[1].text.strip('\n')
        neighborhood = data[2].text.strip('\n')
        if (neighborhood == "Not assigned"):
            neighborhood = borough
        if (borough != "Not assigned"):
            toronto_neighborhoods.append((postalCode, borough, neighborhood))
    except IndexError:pass
df_toronto_neighborhoods = pd.DataFrame([item for item in toronto_neighborhoods])
df_toronto_neighborhoods.columns = ["PostalCode", "Borough", "Neighborhood"]

I'll check the first elements of the resultant dataframe.

In [6]:
df_toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


I'll check the shape of the resultant dataframe.

In [7]:
df_toronto_neighborhoods.shape

(103, 3)

## Week 3 assignment, task 2

It will consist on adding geolocation data to the dataframe previously declared about the neighborhoods of Toronto.

First I would import the libraries required, but these have been imported previously at "Week 3 assignment, task 1" section.

The first real thing to do will be to define a function for getting the latitude and longitude of a postal code from Toronto.

In [8]:
# Define a function for adding the location of a Toronto postal code to the dataframe
def getTorontoPostalCode(row):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(row['PostalCode']))
      lat_lng_coords = g.latlng

    # Extract the coourdinates from the result
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return latitude, longitude

As this function can cost a time applied to all elements of the list, here is a replacement of the previous function using a CSV file.

In [9]:
# Get and process a CSV file with locations based on Toronto postal codes
df_locations_toronto = pd.read_csv("https://cocl.us/Geospatial_data")
df_locations_toronto.set_index("Postal Code", inplace = True)

# Define a function for adding location of a Toronto postal code to the dataframe
def getTorontoPostalCodeCsv(postalCode):
    # Get the coordinates from the CSV file
    latitude = df_locations_toronto.loc[postalCode]['Latitude']
    longitude = df_locations_toronto.loc[postalCode]['Longitude']
    
    return latitude, longitude

Now I'll apply that function to the previously created dataframe in order to add latitude and longitude to the dataframe.

In [10]:
df_toronto_neighborhoods['Latitude'], df_toronto_neighborhoods['Longitude'] = zip(*df_toronto_neighborhoods['PostalCode'].map(getTorontoPostalCodeCsv))

And here is the result:

In [11]:
df_toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Week 3 assignment, task 3

It will consist on analyzing and clustering the neighborhoods of Toronto got from the previous.

First I would import the libraries required, but these have been imported previously at "Week 3 assignment, task 1" section.

The first real thing to do will be to define the credentials and version used for accesing to Foursquare API.

In [12]:
CLIENT_ID = '13NUHFZPOD2RG5D5KTBOKQD3N5OESQHFTL1VZU0YYNBFQT1S' # your Foursquare ID
CLIENT_SECRET = 'NVOGX55INF4QMOVXCHS3PLHTJ45YZT202KKEZBJAWZRCEEVD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
RADIUS = 500 # Radius for the search, worked out based on an average 30 minutes walking distancew

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 13NUHFZPOD2RG5D5KTBOKQD3N5OESQHFTL1VZU0YYNBFQT1S
CLIENT_SECRET:NVOGX55INF4QMOVXCHS3PLHTJ45YZT202KKEZBJAWZRCEEVD


Now I'll define a function for getting the nearby venues of a set of postal codes and names of neighborhoods in Toronto, and their latitudes and longitudes.

In [13]:
def getNearbyVenues(boroughs, neighborhoods, postal_codes, latitudes, longitudes): 
    venues_list=[]
    index = 0
    total = neighborhoods.shape[0]
    for borough, neighborhood, postal_code, lat, lng in zip(boroughs, neighborhoods, postal_codes, latitudes, longitudes): 
        index += 1
        print (str(index) + "/" + str(total) + ": Getting venues from postal code " + postal_code + " ...      ", end="", flush=True)
        print('\r', end='') 
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&raduis={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal_code, 
            borough, 
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'PostalCode',
                  'Borough',  
                  'Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now I'll apply the function with the dates of the dataframe of Toronto neighborhoods in order to create a new dataframe with Toronto venues.

In [14]:
df_toronto_venues_raw = getNearbyVenues(postal_codes=df_toronto_neighborhoods['PostalCode'],
                                   boroughs=df_toronto_neighborhoods['Borough'],
                                   neighborhoods=df_toronto_neighborhoods['Neighborhood'],
                                   latitudes=df_toronto_neighborhoods['Latitude'],
                                   longitudes=df_toronto_neighborhoods['Longitude']
                                  )

Now I'll check the created dataframe with Toronto venues.

In [15]:
df_toronto_venues_raw.head()

,PostalCode,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,North York,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,M3A,North York,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
2,M3A,North York,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
3,M3A,North York,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket
4,M3A,North York,Parkwoods,43.753259,-79.329656,Island Foods,43.745866,-79.346035,Caribbean Restaurant


I'll create another dataframe copy of the venues in order to save the original in case something wanted to be modified without the need to request the info again to Forsquare API.

In [16]:
df_toronto_venues = df_toronto_venues_raw.copy(deep=True)

Checking the resultant venues categories, there is one calle Neighborhoods. Let's check it.

In [17]:
df_toronto_venues[df_toronto_venues['Venue Category'] == 'Neighborhood'].head()

,PostalCode,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
593,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Highland Creek,43.790281,-79.173703,Neighborhood
1178,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Highland Creek,43.790281,-79.173703,Neighborhood
1791,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Highland Creek,43.790281,-79.173703,Neighborhood
2374,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Downtown Toronto,43.653232,-79.385296,Neighborhood
2945,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood


It seems to be a type of venue representing surrounding neighborhoods. I'll drop it.

In [18]:
 df_toronto_venues.drop(df_toronto_venues[df_toronto_venues['Venue Category'] == 'Neighborhood'].index, inplace=True)

Now I'll check the number of venue categories found in Toronto.

In [19]:
df_toronto_venues["Venue Category"].unique().shape

(334,)

A problem for the analysis of the venues in Torono is the huge number of types of categories of the venue (more than 250), because similar venues (like a stadium and a baseball field) will count as different and totally unrelated venues. Checking Foursquare API documentation, there is a hierarchy with up to 4 levels of types of venues, with the API calls providing us the most detailed type. In order to solve this issue, I'll translate the categories of each element of the dataframe of Toronto venues to it's primary category.

For that, I'll ask all types of venues to Foursquare API.

In [20]:
urlCategories = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)

# make the GET request
categories_json = requests.get(urlCategories).json()["response"]

Now I'll create a set of dictionaries for translating each category, no matter the level, to it's primary or secondary category.

In [21]:
def checkPrimaryCategoryByLevel(categoriesList, categoriesDict, primaryCategory):
    for category in categoriesList:
        categoriesDict[category["name"]] = primaryCategory
        if (len(category["categories"]) > 0):
            categoriesDict = checkPrimaryCategoryByLevel(category["categories"], categoriesDict, primaryCategory)
    return categoriesDict

# This is the code for 1st level categories
primaryCategoriesDict = {}
for category in categories_json["categories"]:
    primaryCategoriesDict[category["name"]] = category["name"]
    if (len(category["categories"]) > 0):
        primaryCategoriesDict = checkPrimaryCategoryByLevel(category["categories"], primaryCategoriesDict, category["name"])

# This is the code for 2nd level categories
secondCategoriesDict = {}
for category in categories_json["categories"]:
    secondCategoriesDict[category["name"]] = category["name"]
    if (len(category["categories"]) > 0):
        for second_category in category["categories"]:
            secondCategoriesDict[second_category["name"]] = second_category["name"]
            if (len(second_category["categories"]) > 0):
                secondCategoriesDict = checkPrimaryCategoryByLevel(second_category["categories"], secondCategoriesDict, second_category["name"])
        


And now I'll apply this newly created dictionary over the dataframe of Toronto venues in order to translate venue categories to primary or secondary categories.

In [22]:
def translateVenueToPrimaryCategory(row):
    return primaryCategoriesDict[row["Venue Category"]]

def translateVenueToSecondaryCategory(row):
    return secondCategoriesDict[row["Venue Category"]]

df_toronto_venues["Venue Category"] = df_toronto_venues.apply(translateVenueToSecondaryCategory, axis=1)

We can check the categories translation has been done correctly by checking again the number of categories, this time smaller and easier to manage (less than 10 in case of primary categories, over 200 in case of secondary categories).

In [23]:
df_toronto_venues["Venue Category"].unique().shape

(215,)

Now I'll transform the column "Venue Category" into multiple boolean columns through one-hot encoding

In [24]:
df_toronto_venues_onehot = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix="", prefix_sep="")
df_toronto_venues_onehot['PostalCode'] = df_toronto_venues['PostalCode'] 
df_toronto_venues_onehot['Venue'] = df_toronto_venues['Venue'] 
df_toronto_venues_onehot = df_toronto_venues.merge(df_toronto_venues_onehot, on=['PostalCode', 'Venue'])
df_toronto_venues_onehot.drop(['Venue Category'], axis=1, inplace=True)

Now I'll check the resultant Toronto venues dataframe with one-hot encoding:

In [25]:
df_toronto_venues_onehot.head()

,PostalCode,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Afghan Restaurant,African Restaurant,...,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Warehouse Store,Wings Joint,Zoo
0,M3A,North York,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,North York,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,North York,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M3A,North York,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M3A,North York,Parkwoods,43.753259,-79.329656,Tim Hortons,43.752814,-79.314067,0,0,...,0,0,0,0,0,0,0,0,0,0


Now I'll update a copy of the Toronto neighborhoods dataframe by taking the mean of the frequency of occurrence of each category. I'll will also add a column with the number of venues at each neighborhood. 

In [26]:
df_toronto_neighborhoods_freq_venues = df_toronto_venues_onehot.groupby('PostalCode').mean().reset_index()
df_toronto_neighborhoods_extended = df_toronto_neighborhoods.copy(deep=True)
df_toronto_neighborhoods_extended = df_toronto_neighborhoods_extended.merge(
    df_toronto_venues_onehot.groupby('PostalCode').count().reset_index()[['PostalCode', 'Borough']].rename(columns={"Borough": "Total Venues"}),
    on='PostalCode')
df_toronto_neighborhoods_extended = df_toronto_neighborhoods_extended.merge(df_toronto_neighborhoods_freq_venues, on='PostalCode')
df_toronto_neighborhoods_extended.drop(['Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Latitude', 'Venue Longitude'], axis=1, inplace=True)

Now I'll check the resultant Toronto neighborhoods dataframe:

In [27]:
df_toronto_neighborhoods_extended.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Total Venues,Afghan Restaurant,African Restaurant,Airport,American Restaurant,...,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Warehouse Store,Wings Joint,Zoo
0,M3A,North York,Parkwoods,43.753259,-79.329656,120,0.000000,0.008333,0.000000,0.016667,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,106,0.009434,0.009434,0.000000,0.018868,...,0.0,0.0,0.018868,0.0,0.009434,0.0,0.00000,0.009434,0.0,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,104,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,106,0.000000,0.000000,0.009434,0.018868,...,0.0,0.0,0.018868,0.0,0.000000,0.0,0.00000,0.009434,0.0,0.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,123,0.000000,0.008130,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00813,0.000000,0.0,0.0


Now I'll cluster the neighborhoods using the K-Means algorithm.The clusterization will be based on the frequency of the different types of venues. different number of clusters will be test in order to get the number of clusters with better results.

In [28]:
# set number of clusters
kclusters = 5

df_toronto_neighborhoods_clustering = df_toronto_neighborhoods_extended.drop(['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude', 'Total Venues'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_toronto_neighborhoods_clustering)

Now check the resultant labels:

In [29]:
kmeans.labels_

array([1, 1, 1, 1, 3, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 4, 1, 1, 0, 4, 4, 1,
       3, 1, 1, 4, 0, 2, 3, 1, 4, 4, 3, 0, 3, 4, 1, 4, 1, 2, 1, 4, 4, 4,
       1, 1, 3, 4, 3, 1, 3, 0, 2, 1, 4, 1, 1, 0, 1, 2, 3, 3, 3, 4, 0, 0,
       1, 1, 3, 4, 0, 1, 2, 1, 4, 4, 0, 0, 2, 1, 4, 4, 2, 4, 4, 2, 1, 4,
       1, 3, 2, 1, 1, 1, 3, 0, 4, 4, 1, 1, 4, 4, 1], dtype=int32)

Now add the labels to a copy of the Toronto neighborhoods dataframe.

In [30]:
df_toronto_neighborhoods_clustered = df_toronto_neighborhoods_extended.copy(deep=True)
df_toronto_neighborhoods_clustered.insert(6, 'Cluster Label', kmeans.labels_)
df_toronto_neighborhoods_clustered.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Total Venues,Cluster Label,Afghan Restaurant,African Restaurant,Airport,...,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Warehouse Store,Wings Joint,Zoo
0,M3A,North York,Parkwoods,43.753259,-79.329656,120,1,0.000000,0.008333,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,106,1,0.009434,0.009434,0.000000,...,0.0,0.0,0.018868,0.0,0.009434,0.0,0.00000,0.009434,0.0,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,104,1,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,106,1,0.000000,0.000000,0.009434,...,0.0,0.0,0.018868,0.0,0.000000,0.0,0.00000,0.009434,0.0,0.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,123,3,0.000000,0.008130,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00813,0.000000,0.0,0.0


Let's analyze the characteristics of each cluster by getting the mean of the frequency of each type of venue inside the cluster and printing the top 5 venues at each cluster:

In [33]:
num_top_venues = 5
df_toronto_clusters = df_toronto_neighborhoods_clustered.groupby('Cluster Label').mean().reset_index()
df_toronto_clusters.drop(['Latitude', 'Longitude'], axis=1, inplace=True)
for cluster in range(kclusters):
    print("---- Cluster " + str(cluster) + " ----")
    print("Average number of venues: " + str(round(df_toronto_clusters[df_toronto_clusters['Cluster Label'] == cluster].iloc[0]['Total Venues'])))
    print("Neighborhoods in the cluster: " + str(df_toronto_neighborhoods_clustered[df_toronto_neighborhoods_clustered['Cluster Label'] == cluster].shape[0]))
    temp = df_toronto_clusters[df_toronto_clusters['Cluster Label'] == cluster].drop(['Total Venues'], axis=1).T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Cluster 0 ----
Average number of venues: 166
Neighborhoods in the cluster: 15
               venue  freq
0        Coffee Shop  0.16
1           Pharmacy  0.10
2     Sandwich Place  0.07
3               Bank  0.07
4  Food & Drink Shop  0.07


---- Cluster 1 ----
Average number of venues: 115
Neighborhoods in the cluster: 39
                venue  freq
0         Coffee Shop  0.15
1   Food & Drink Shop  0.08
2    Asian Restaurant  0.07
3  Athletics & Sports  0.04
4        Dessert Shop  0.03


---- Cluster 2 ----
Average number of venues: 110
Neighborhoods in the cluster: 9
               venue  freq
0   Asian Restaurant  0.22
1        Coffee Shop  0.10
2  Food & Drink Shop  0.09
3               Bank  0.05
4           Pharmacy  0.03


---- Cluster 3 ----
Average number of venues: 185
Neighborhoods in the cluster: 14
                venue  freq
0         Coffee Shop  0.40
1      Sandwich Place  0.05
2    Asian Restaurant  0.05
3  Athletics & Sports  0.03
4   Food & Drink Shop  0.03


-

Based on the information shown previously, these are the clusters:

- Cluster 0: Many venues, most of them coffee shops and pharmacies.
- Cluster 1: Few venues, mostly for shopping and transportation.
- Cluster 2: Few venues
- Cluster 3: 
- Cluster 4:  


Finally, show the neighborhoods and the clusters assigned to each neighborhood in a map:

In [32]:
# create Toronto map
latitude = 43.651070
longitude = -79.347015
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, venues, cluster in zip(df_toronto_neighborhoods_clustered['Latitude'], 
                                           df_toronto_neighborhoods_clustered['Longitude'], 
                                           df_toronto_neighborhoods_clustered['Neighborhood'],
                                           df_toronto_neighborhoods_clustered['Total Venues'], 
                                           df_toronto_neighborhoods_clustered['Cluster Label']):
    label = folium.Popup(str(neighborhood) + '; ' + str(venues) + ' venues; Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters